In [1]:
#from __future__ import unicode_literals, print_function, division
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, CuDNNLSTM, CuDNNGRU, TimeDistributed, Flatten, Dropout, Bidirectional, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint
from numpy import array, argmax
from numpy.random import rand, shuffle
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from keras.utils import multi_gpu_model, to_categorical
from keras.utils.vis_utils import plot_model
import scipy
import statsmodels
import sklearn
import tensorflow as tf
tf.enable_eager_execution()
import keras
from io import open
import unicodedata
import random
import math
import os
import time
import re
import sys
from unicodedata import normalize
from numpy import array
from pickle import dump, load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint

INDIC_NLP_LIB_HOME=r"anoopkunchukuttan-indic_nlp_library-eccde81/src"
INDIC_NLP_RESOURCES=r"indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

Using TensorFlow backend.
anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/indic_scripts.py:116: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ALL_PHONETIC_VECTORS= ALL_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()
anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/indic_scripts.py:117: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  TAMIL_PHONETIC_VECTORS=TAMIL_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()
anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/english_script.py:113: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ENGLISH_PHONETIC_VECTORS=ENGLISH_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()


In [2]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

def max_length(tensor):
    return max(len(t) for t in tensor)

In [3]:
def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

def to_pairs(english_text, hindi_text):
    english_lines = english_text.strip().split('\n')
    hindi_lines = hindi_text.strip().split('\n')
    pairs = []
    for i in range(len(hindi_lines)):
        pairs.append([])
        pairs[i].append(pre_process_english_sentence(english_lines[i]))
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
    return pairs

def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text=text.replace("'", "")
    text=text.replace('"', "")
    text= re.sub("'", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

def pre_process_english_sentence(line):
    line = line.lower()
    line = clean_text(line)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [re_print.sub('', w) for w in line]
    line = [word for word in line if word.isalpha()]
    line = ' '.join(line)
    return line

def pre_process_hindi_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi",remove_nuktas)
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    return (line)

english_text = load_doc('English')
hindi_text = load_doc('Hindi')
pairs = to_pairs(english_text, hindi_text)

In [4]:
n_sentences = len(pairs)
n_train = (int)(n_sentences/2)

In [5]:
final_pairs = []

for i in range(n_train):
    final_pairs.append([])
    final_pairs[i].append(pairs[i][0])
    final_pairs[i].append(pairs[i][1])

length_till = len(final_pairs)
for i in range((int)(0.1*n_train)):
    final_pairs.append([])
    final_pairs[i+length_till].append(pairs[i+n_train][0])
    final_pairs[i+length_till].append(pairs[i+n_train][1])

length_train = len(final_pairs)
length_till = len(final_pairs)
for i in range((int)(0.1*n_train)):
    final_pairs.append([])
    final_pairs[i+length_till].append(pairs[n_sentences-1-i][0])
    final_pairs[i+length_till].append(pairs[n_sentences-1-i][1])

len_consider = len(final_pairs)

In [6]:
test_final_pairs = []
for i in range(n_sentences - len_consider):
    test_final_pairs.append([])
    test_final_pairs[i].append(pairs[i+length_till][0])
    test_final_pairs[i].append(pairs[i+length_till][1])
    
print (len(test_final_pairs))
print (len(final_pairs))
print (len(pairs))

19960
29936
49896


In [7]:
final_training_pairs = []

for i in range((int)(0.1*n_train)):
    final_training_pairs.append([])
    final_training_pairs[i].append(pairs[i+n_train][0])
    final_training_pairs[i].append(pairs[i+n_train][1])

length_till = len(final_training_pairs)
for i in range((int)(0.1*n_train)):
    final_training_pairs.append([])
    final_training_pairs[i+length_till].append(pairs[n_sentences-1-i][0])
    final_training_pairs[i+length_till].append(pairs[n_sentences-1-i][1])
    
print (len(final_training_pairs))
final_training_pairs = final_training_pairs + final_pairs
print ((int)(0.2*n_train))
print (len(final_training_pairs))

4988
4989
34924


In [8]:
with open("traing_english.txt", "w") as f:
    for i in range(len(final_training_pairs)):
        f.write(str(final_training_pairs[i][0]) +"\n")

with open("traing_hindi.txt", "w") as f:
    for i in range(len(final_training_pairs)):
        f.write(str(final_training_pairs[i][1]) +"\n")
        
with open("testing_english.txt", "w") as f:
    for i in range(len(test_final_pairs)):
        f.write(str(test_final_pairs[i][0]) +"\n")

with open("testing_hindi.txt", "w") as f:
    for i in range(len(test_final_pairs)):
        f.write(str(test_final_pairs[i][1]) +"\n")